# Checking your code with unit tests

In software development, **unit tests** are allowed to verify that the code works as desired.
A unit test is a bit like an answer sheet for an exam: somebody works out the correct answers in advance for specific inputs, and then one checks that the code does indeed produce these answers.
If the code deviates from the predetermined answers, the error is pointed out so that the programmer can see where the code goes wrong.
Careful, though: unit tests can only tell you **where** your code produces wrong answers, not **why** it produces wrong answers.
That's still for you to figure out.

## A toy example

Suppose we want to implement our own addition operation as follows.

In [ ]:
def addition(m, n):
    """And m and n in a roundabout manner."""
    return (m - 1) + (n + 1)

In order to check the code for correctness, we can now just run it on various values for `m` and `n`.
So our unit test might look something like this:

| `m` | `n` | sum |
| --: | --: | --: |
| 0   | 0   | 0   |
| 137 | -5  | 132 |
| 2   | 1000000 | 1000002 |

In a sense, this is exactly what you do anyways when you test your own code: provide some example inputs and make sure that the function returns what you want it to return.
In practice, though, the whole points of unit tests is that you do not have to do things by hand.
Instead, the whole testing process is automated so that hundreds and thousands of tests can be run in a few seconds.

**Caution:**
Passing a unit test is not a guarantee that your code works flawlessly.
Somebody had to design those unit tests, after all, so if they made a mistake in the tests or forgot something, something might slip past the test.
It's similar to an exam: just because you got an A does not mean that you could solve every conceivable assignment.
It just means that you succeeded on the given assignments, the implicit assumption being that these assignments are a thorough test of your understanding.

## Unit testing on CoCalc

All future homeworks will come with predefined unit tests.
This allows you to check your homework solutions for correctness before they get handed in.
In order to use them, you have to follow a specific order of steps.

1. **Use functions**  
   First, all your code has to take the form of functions.
   That's because only functions can be automatically unit-tested.
   So whatever code you want to test, make sure it only consists of function definition.
1. **Name your functions correctly**  
   Second, your functions must have the correct name.
   Each homework will tell you the names of crucial functions.
   If you deviate from these names by even one character, the unit tests won't be able to find the function they're supposed to test
1. **Write to student_solutions.py**  
   All functions must be written to a file `student_solutions.py`.
   This is very easy, and will be explained in a moment.
1. **Run the unit tests**  
   This is also very easy and will be explained in a second.

Let's work through a concrete example.
Suppose your assignment is to reimplement addition and subtraction.
The function names you are supposed to use are `addition` and `subtraction` (obvious choices, hmm?).
One student, *Crazy C*, comes up with the following solution.

In [ ]:
def addition(m, n):
    return m + n  # so far, so good


# but here things get weird
def subtraction(m, n):
    return tripled(m) - tripled(n)


def tripled(n):
    return 3 * n

print(addition(5, 3))
print(subtraction(1, 1))

Crazy C has done a few things right:
All the important code takes the form of function definitions, and two of the functions have the requested names `addition` and `subtraction`.
We're not too happy about the missing docstrings, but we'll ignore this for now.

Crazy C now has to test this code.
At the bottom of the notebook with the assignment, there is a code cell with the following text:

```python
%% writefile tests/student_solutions.py
# copy-paste your code here, then run the cell
```

Crazy C does as instructed.

In [ ]:
%%writefile tests/student_solutions.py
# copy-paste your code here, then run the cell

def addition(m, n):
    return m + n

def subtraction(m, n):
    return tripled(m) - tripled(n)

This creates a new file called `student_solutions.py` in the subfolder `tests`.
The file contains the contents of the cell.
Without this file, the unit tests won't work because we can't directly run unit tests on a Jupyter notebook.
Instead, we have to write all the code to a separate Python file.

Next, there is a cell that looks as follows:

In [ ]:
# test addition
!pytest-3 tests/addition.py

Crazy C runs the cell, and after some waiting there is an output `1 passed`, informing her that the addition test has been passed.

Right below, there is another cell - this one seems to be for subtraction.

In [ ]:
!pytest-3 tests/subtraction.py

Running this example gives a lot more output.
But the essential part is at the bottom: `1 failed`.
The unit test was not passed correctly.
Why is that?
Looking a little bit higher, we see `NameError: name 'tripled' is not defined`.

Upon reflection, Crazy C realizes what's wrong: not all the relevant code was copy-pasted into the cell with the `%%writefile` line.
Since the function `subtraction` uses the function `tripled`, the code for this also must be copied into the cell.
Time to update the cell and run it again:

In [ ]:
%%writefile tests/student_solutions.py
# copy-paste your code here, then run the cell

def addition(m, n):
    return m + n

def subtraction(m, n):
    return tripled(m) - tripled(n)

def tripled(n):
    return 3 * n

Okay, the old version of `student_solutions` has been overwritten with the new code.
Time to run the subtraction test again.

In [ ]:
!pytest-3 tests/subtraction.py

Still no dice.
The `NameError` has disappeared, but it has been replaced by an `AssertionError`.
These errors show up when the student code can be run, but does not produce the correct output.
What is the problem here?
The highlighted lines show the problem:

```python
assert(1 - 0) == 3
 +  where 3 = subtraction(1, 0)
```

When the first argument for `subtraction` is `1` and the second argument is `0`, `subtraction` returns 3.
But obviously the correct solution for `1 - 0` is `1`, not `3`.
Crazy C is confused at first, but decides after a moment of clarity that the problem is with the use of `tripled`.
So once again the code is modified, written to a file, and the test repeated.

In [ ]:
%%writefile tests/student_solutions.py
# copy-paste your code here, then run the cell

def addition(m, n):
    return m + n

def subtraction(m, n):
    return m - n

In [ ]:
!pytest-3 tests/subtraction.py

Finally, the code passes the subtraction test.
Since all tests have been passed correctly, the homework is ready for collection.

## When and what to test

### Test early and often

Don't wait with running tests until the end of the assignment.
As soon as you have finished a function, test it.
If it performs correctly, you know for a fact that you have a solid base to build on.
If your function doesn't pass the test, you can fix it right away.
Waiting too long with fixing a function means that you might have to rewrite a lot of code that was designed for an earlier version of the function.
And the larger your code base, the longer it will take to figure out what's wrong.

### Test everything to avoid regressions

After a test fails, you'll have to modify your code.
Once you've made your changes, make sure to rerun the whole test suite.
Otherwise, you might miss **regressions**.
A regression occurs when your code suddenly produces mistakes it didn't make before.
Basically, by trying to plug one hole in your code you might have accidentally created a new one.
If you only run the tests that have failed in a previous attempt, any new bugs you might have introduced could go unnoticed.
So always test all your pieces of code that you have already written, not just the ones that failed a test earlier on.

## Bullet-point summary

- Unit tests can automatically test code for correctness.
  A unit test ensures that certain inputs match the intended output.
- Passing a unit test is not a perfect guarantee of correctness.
  There might be edge cases that the test did not check for.
- For homeworks, follow the standard procedure:
  1. Copy-paste your code into the cell with ``%%writefile tests/student_solutions.py``.
  1. Make sure to include all relevant functions.
  1. Make sure all functions are named correctly.
  1. Run the tests.
     If a test fails, modify your code and repeat from the step 1.
- Test early and often.
- Always run all tests after making changes to avoid **regressions**.